In [1]:
import os
os.environ["MANTLE"] = "lattice"
import silica as si
import magma as m

@si.coroutine(inputs={"I": si.Bit})
def TFF(init=False):
    value = init
    I = yield
    while True:
        O = value
        value = I ^ value
        I = yield O

In [2]:
from loam.boards.icestick import IceStick
from mantle import Counter

icestick = IceStick()
icestick.Clock.on()
for i in range(len(icestick.J1)):
    icestick.J1[i].output().on()
main = icestick.main()
tff = si.compile(TFF())()
counter = Counter(2)
m.wire(counter.O[1], tff.I)
m.wire(tff.O, main.J1[0])
wire(m.bits(0, 7), main.J1[1:])

m.compile("build/silica_tff", main)

import mantle lattice ice40
import mantle lattice mantle40
compiling Register3CE_0001
compiling Or3
compiling Or3x3
compiling __silica_BufferTFF
compiling And2
compiling And2x3
compiling XOr2
compiling TFF
compiling FullAdder
compiling Add2Cout
compiling Register2
compiling Counter2
compiling main


In [3]:
%%bash
yosys -q -p 'synth_ice40 -top main -blif build/silica_tff.blif' build/silica_tff.v
arachne-pnr -d 1k -o build/silica_tff.txt -p build/silica_tff.pcf build/silica_tff.blif
icepack build/silica_tff.txt build/silica_tff.bin
icetime -tmd hx1k build/silica_tff.txt

// Reading input .asc file..
// Reading 1k chipdb file..
// Creating timing netlist..

icetime topological timing analysis report

Info: max_span_hack is enabled: estimate is conservative.

Report for critical path:
-------------------------

        lc40_6_13_7 (LogicCell40) [clk] -> lcout: 0.640 ns
     0.640 ns net_11750 (inst0.inst0.inst0_Q)
        t33 (LocalMux) I -> O: 0.330 ns
        inmux_6_13_13900_13944 (InMux) I -> O: 0.260 ns
        lc40_6_13_3 (LogicCell40) in0 -> lcout: 0.449 ns
     1.678 ns net_11746 (inst0.inst0.inst2_O)
        t29 (LocalMux) I -> O: 0.330 ns
        inmux_7_13_16013_16047 (InMux) I -> O: 0.260 ns
        lc40_7_13_2 (LogicCell40) in0 -> lcout: 0.449 ns
     2.716 ns net_13854 (inst0.inst12.inst0_O)
        t47 (LocalMux) I -> O: 0.330 ns
        inmux_7_13_16004_16071 (InMux) I -> O: 0.260 ns
        lc40_7_13_6 (LogicCell40) in0 -> lcout: 0.449 ns
     3.754 ns net_13858 (inst0.inst11.inst0_O)
        odrv_7_13_13858_9781 (Odrv4) I -> O: 0.372 ns

seed: 1
device: 1k
read_chipdb +/share/arachne-pnr/chipdb-1k.bin...
  supported packages: cb121, cb132, cb81, cm121, cm36, cm49, cm81, qn84, swg16tr, tq144, vq100
read_blif build/silica_tff.blif...
prune...
read_pcf build/silica_tff.pcf...
instantiate_io...
pack...

After packing:
IOs          9 / 96
GBs          0 / 8
  GB_IOs     0 / 8
LCs          27 / 1280
  DFF        6
  CARRY      0
  CARRY, DFF 0
  DFF PASS   0
  CARRY PASS 0
BRAMs        0 / 16
WARMBOOTs    0 / 1
PLLs         0 / 1

place_constraints...
promote_globals...
  promoted CLKIN$2, 6 / 6
  promoted 1 nets
    1 clk
  1 globals
    1 clk
realize_constants...
  realized 1
place...
  initial wire length = 108
  at iteration #50: temp = 3.58845, wire length = 159
  at iteration #100: temp = 0.995399, wire length = 54
  at iteration #150: temp = 0.0964593, wire length = 17
  final wire length = 17

After placement:
PIOs       5 / 96
PLBs       4 / 160
BRAMs      0 / 16

  place time 0.03s
route...
  pass 1, 0 shared.

Aft

In [4]:
from mantle import DFF

TFF = m.DefineCircuit('TFF', 'I', m.In(m.Bit), 'O', m.Out(m.Bit), 'CLK', m.In(m.Bit))

# create ff holding state first
ff = DFF()

# compute the next state
ff( TFF.I ^ ff.O )

m.wire(ff.O, TFF.O)
m.wire(TFF.CLK, ff.CLK)

m.EndDefine()


icestick = IceStick()
icestick.Clock.on()
for i in range(len(icestick.J1)):
    icestick.J1[i].output().on()
main = icestick.main()
tff = TFF()
counter = Counter(2)
m.wire(counter.O[1], tff.I)
m.wire(tff.O, main.J1[0])
wire(m.bits(0, 7), main.J1[1:])

m.compile("build/magma_tff", main)

compiling XOr2
compiling TFF
compiling FullAdder
compiling Add2Cout
compiling Register2
compiling Counter2
compiling main
Warning (verilog): input CLK not connected to an output


In [5]:
%%bash
yosys -q -p 'synth_ice40 -top main -blif build/magma_tff.blif' build/magma_tff.v
arachne-pnr -d 1k -o build/magma_tff.txt -p build/magma_tff.pcf build/magma_tff.blif
icepack build/magma_tff.txt build/magma_tff.bin
icetime -tmd hx1k build/magma_tff.txt

// Reading input .asc file..
// Reading 1k chipdb file..
// Creating timing netlist..

icetime topological timing analysis report

Info: max_span_hack is enabled: estimate is conservative.

Report for critical path:
-------------------------

        pre_io_0_8_1 (PRE_IO) [clk] -> DIN0: 0.240 ns
     0.240 ns net_836 (CLKIN$2)
        odrv_0_8_836_886 (Odrv12) I -> O: 0.540 ns
        t3 (Sp12to4) I -> O: 0.449 ns
        t2 (Span4Mux_v4) I -> O: 0.372 ns
        t1 (Span4Mux_v4) I -> O: 0.372 ns
        t0 (LocalMux) I -> O: 0.330 ns
        inmux_8_14_18234_18314 (ClkMux) I -> O: 0.309 ns
     2.611 ns net_18314 (CLKIN$2)
        lc40_8_14_7 (LogicCell40) clk [setup]: 0.000 ns
     2.611 ns net_16091 (inst1.inst0.inst1.I0)

Resolvable net names on path:
     0.240 ns ..  2.611 ns CLKIN$2
                  lcout -> inst1.inst0.inst1.I0

Total number of logic levels: 1
Total path delay: 2.61 ns (383.02 MHz)



seed: 1
device: 1k
read_chipdb +/share/arachne-pnr/chipdb-1k.bin...
  supported packages: cb121, cb132, cb81, cm121, cm36, cm49, cm81, qn84, swg16tr, tq144, vq100
read_blif build/magma_tff.blif...
prune...
read_pcf build/magma_tff.pcf...
instantiate_io...
pack...

After packing:
IOs          9 / 96
GBs          0 / 8
  GB_IOs     0 / 8
LCs          3 / 1280
  DFF        3
  CARRY      0
  CARRY, DFF 0
  DFF PASS   0
  CARRY PASS 0
BRAMs        0 / 16
WARMBOOTs    0 / 1
PLLs         0 / 1

place_constraints...
promote_globals...
  promoted 0 nets
  0 globals
realize_constants...
  realized 1
place...
  initial wire length = 73
  at iteration #50: temp = 5.77093, wire length = 34
  at iteration #100: temp = 1.86709, wire length = 27
  at iteration #150: temp = 0.0756508, wire length = 21
  final wire length = 21

After placement:
PIOs       5 / 96
PLBs       2 / 160
BRAMs      0 / 16

  place time 0.00s
route...
  pass 1, 0 shared.

After routing:
span_4     7 / 6944
span_12    1 / 1440
